<a href="https://colab.research.google.com/github/Tulipraaj/FoCID-Forecasting_Cryptocurrency_Investment_Decision/blob/main/SVM_Bagging__and_Boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import numpy as np
import math as m
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error ,r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_csv("BTC.csv")

In [4]:
df.head()

,date,block_size,difficulty,hashrate,market_cap,mining_profitability,transaction,tweets,price
0,01-01-2018,966169,1.922580e+12,1.500000e+19,2.266610e+11,2.174,241601,62044.0,13523
1,02-01-2018,968332,1.931140e+12,1.600000e+19,2.334150e+11,2.431,340809,77723.0,13924
2,03-01-2018,975882,1.931140e+12,1.490000e+19,2.519290e+11,2.737,395806,79086.0,15027
3,04-01-2018,975713,1.931140e+12,1.640000e+19,2.495520e+11,2.606,424840,74534.0,14883
4,05-01-2018,974894,1.931140e+12,1.500000e+19,2.663230e+11,2.631,342564,76404.0,15881


In [5]:

df.tail()

,date,block_size,difficulty,hashrate,market_cap,mining_profitability,transaction,tweets,price
1821,27-12-2022,706675,3.536410e+13,2.120000e+20,3.234390e+11,0.0589,239244,85632.0,16807
1822,28-12-2022,542347,3.536410e+13,2.530000e+20,3.202390e+11,0.0704,272781,107117.0,16640
1823,29-12-2022,692913,3.536410e+13,2.740000e+20,3.193230e+11,0.0545,265814,96225.0,16591
1824,30-12-2022,725552,3.536410e+13,2.540000e+20,3.185390e+11,0.0658,290857,97462.0,16550
1825,31-12-2022,573296,3.536410e+13,2.940000e+20,3.189440e+11,0.0557,242169,77382.0,16570


In [6]:
df.isna().sum()

date                     0
block_size               0
difficulty               0
hashrate                 0
market_cap               0
mining_profitability     0
transaction              0
tweets                  37
price                    0
dtype: int64

In [7]:
df.dtypes

date                     object
block_size                int64
difficulty              float64
hashrate                float64
market_cap              float64
mining_profitability    float64
transaction               int64
tweets                  float64
price                     int64
dtype: object

In [8]:
df.fillna(method='ffill', inplace=True)


In [9]:
df.isna().sum()

date                    0
block_size              0
difficulty              0
hashrate                0
market_cap              0
mining_profitability    0
transaction             0
tweets                  0
price                   0
dtype: int64

In [10]:
X = df[['tweets','market_cap','difficulty','hashrate','mining_profitability']]
# Assuming 'Close' is the target variable
y = df['price']

In [11]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [12]:
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


MODEL 1


In [13]:
svm_model = SVR(kernel='linear', C=10, epsilon=0.1)


In [16]:
bagging_model = BaggingRegressor(base_estimator=svm_model, n_estimators=10, random_state=42)
bagging_model.fit(X_train, y_train)
y_pred_bagging = bagging_model.predict(X_test)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [17]:
boosting_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
boosting_model.fit(X_train, y_train)
y_pred_boosting = boosting_model.predict(X_test)

In [18]:
def print_metrics(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mpe = np.mean((y_true - y_pred) / y_true) * 100
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    r2 = r2_score(y_true, y_pred)

    print(f"Metrics for {model_name}:")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"MPE: {mpe}")
    print(f"MAPE: {mape}")
    print(f"R2 Score: {r2}")
    print()

print_metrics(y_test, y_pred_bagging, "Bagging")
print_metrics(y_test, y_pred_boosting, "Boosting")


Metrics for Bagging:
MSE: 77181230.1324766
RMSE: 8785.284863479192
MAE: 5215.295148234895
MPE: 3.2077375454159034
MAPE: 18.3675568988987
R2 Score: 0.7494880486490871

Metrics for Boosting:
MSE: 40510.59080976042
RMSE: 201.27242933337993
MAE: 140.3891191303844
MPE: -0.013372130131045106
MAPE: 0.8749575621547281
R2 Score: 0.9998685122388343

